In [246]:
PROJ_PATH = '/home/andrea/Projects/piacere-model-checker'
TESTS_PATH = PROJ_PATH + '/tests/doml/CaseStudies/*.domlx'

In [247]:
from glob import glob

In [248]:
domlx_files = glob(TESTS_PATH)

domlx_files

['/home/andrea/Projects/piacere-model-checker/tests/doml/CaseStudies/SIMPA_Validation_v2.domlx',
 '/home/andrea/Projects/piacere-model-checker/tests/doml/CaseStudies/nio3_test_exec_env.domlx',
 '/home/andrea/Projects/piacere-model-checker/tests/doml/CaseStudies/posidonia.domlx',
 '/home/andrea/Projects/piacere-model-checker/tests/doml/CaseStudies/posidonia_IOP.domlx',
 '/home/andrea/Projects/piacere-model-checker/tests/doml/CaseStudies/uc3-openstack_v2.domlx']

In [249]:
import subprocess
import os
from time import time
import pandas as pd

os.chdir(PROJ_PATH)

print(os.getcwd())

test_cmd = lambda file: f"python -m mc_openapi -z -d {file}"

df = pd.DataFrame(columns=['filename', 'time', 'doml_ver', 'result', '#error', 'memory', 'conflicts', 'quant inst'])

for file in domlx_files:

    filename = file.split("/")[-1]

    # I know that using time is inaccurate, but it's simple and it works for generating simple benchmarks
    start_time = time()
    out = subprocess.run(test_cmd(file), shell=True, capture_output=True)
    run_time = time() - start_time

    r = out.stdout.decode('utf-8')

    print(r)

    doml_ver, _, r = r.rpartition("[RESULT]")
    result, _, r = r.rpartition("[ERRORS]")
    errors, _, stats = r.rpartition("[STATS]")
    _, _, doml_ver = doml_ver.rpartition("v")

    doml_ver = doml_ver.replace("\n", "")
    result = result.replace("\n", "")
    error_num = len(errors.split("\n")) - 1
    stats = stats.split("\n")[2].split(",")[-3:]

    df2 = pd.DataFrame([[filename, run_time, doml_ver, result, error_num, *stats]], columns=df.columns)

    df = pd.concat([df, df2])

df

/home/andrea/Projects/piacere-model-checker
Using DOML v2.2.1
[RESULT]
unsat
[ERRORS]
[Built-in]
Software component 'nio3_git' is not deployed to any abstract infrastructure node.
[STATS]
memory,conflicts,quant instantiations
40.38,1273,1253


Using DOML v2.2.1
[RESULT]
sat
[STATS]
memory,conflicts,quant instantiations
43.29,2592,2614


Using DOML v2.2.1
[RESULT]
unsat
[ERRORS]
[Built-in]
Virtual machine gestaut_vm is connected to no network interface.
Software components 'Database' and 'Gestaut' are supposed to communicate through interface 'dbAccess', but they are deployed to nodes that cannot communicate through a common network.
Abstract infrastructure element 'gestaut_vm' has not been mapped to any element in the active concretization.
Security group 'sg' is not associated with any network interface.
[STATS]
memory,conflicts,quant instantiations
91.13,1875,2794


Using DOML v2.2.1
[RESULT]
unsat
[ERRORS]
[Built-in]
Virtual machine gestaut_vm is connected to no network interface.
S

,filename,time,doml_ver,result,#error,memory,conflicts,quant inst
0,SIMPA_Validation_v2.domlx,1.344904,2.2.1,unsat,3,40.38,1273,1253
0,nio3_test_exec_env.domlx,1.602703,2.2.1,,2,43.29,2592,2614
0,posidonia.domlx,16.781076,2.2.1,unsat,6,91.13,1875,2794
0,posidonia_IOP.domlx,16.956536,2.2.1,unsat,6,91.45,1417,2263
0,uc3-openstack_v2.domlx,10.553367,2.2.1,unsat,3,91.29,7649,8422
